# <center>**MODELISATION PREDICTIVE DANS R AVEC LE PACKAGE CARET : REGRESSION**

<center>Par Josué AFOUDA

# <font color=red> Importation des packages et des données

La librairie [caret](https://www.rdocumentation.org/packages/caret/versions/4.47) est l'un des packages de R les plus utilisés en apprentissage automatique supervisé. C'est principalement cette librairie que nous utiliserons dans cette formation.

In [1]:
# Packages

library(caret)

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


L'ensemble de données que nous utiliserons contient des informations sur les caractéristiques de certaines maisons dans la région de Boston. Il s’agira de construire un modèle de régression linéaire capable de prédire le prix d’une maison en fonction de certains de ses attributs.

In [2]:
# Importation des données

boston <- read.csv('https://raw.githubusercontent.com/JosueAfouda/Machine-Learning-par-la-pratique-avec-Python/master/Boston.csv')

head(boston)

CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2
0.02985,0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,5.21,28.7


In [3]:
# Structure des données

str(boston)

'data.frame':	506 obs. of  13 variables:
 $ CRIM   : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ ZN     : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ INDUS  : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ CHAS   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ NOX    : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ RM     : num  6.58 6.42 7.18 7 7.15 ...
 $ AGE    : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ DIS    : num  4.09 4.97 4.97 6.06 6.06 ...
 $ RAD    : int  1 2 2 3 3 3 5 5 5 5 ...
 $ TAX    : int  296 242 242 222 222 222 311 311 311 311 ...
 $ PTRATIO: num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ LSTAT  : num  4.98 9.14 4.03 2.94 5.33 ...
 $ MEDV   : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...


In [4]:
# Résumé statistique

summary(boston)

      CRIM                ZN             INDUS            CHAS        
 Min.   : 0.00632   Min.   :  0.00   Min.   : 0.46   Min.   :0.00000  
 1st Qu.: 0.08204   1st Qu.:  0.00   1st Qu.: 5.19   1st Qu.:0.00000  
 Median : 0.25651   Median :  0.00   Median : 9.69   Median :0.00000  
 Mean   : 3.61352   Mean   : 11.36   Mean   :11.14   Mean   :0.06917  
 3rd Qu.: 3.67708   3rd Qu.: 12.50   3rd Qu.:18.10   3rd Qu.:0.00000  
 Max.   :88.97620   Max.   :100.00   Max.   :27.74   Max.   :1.00000  
      NOX               RM             AGE              DIS        
 Min.   :0.3850   Min.   :3.561   Min.   :  2.90   Min.   : 1.130  
 1st Qu.:0.4490   1st Qu.:5.886   1st Qu.: 45.02   1st Qu.: 2.100  
 Median :0.5380   Median :6.208   Median : 77.50   Median : 3.207  
 Mean   :0.5547   Mean   :6.285   Mean   : 68.57   Mean   : 3.795  
 3rd Qu.:0.6240   3rd Qu.:6.623   3rd Qu.: 94.08   3rd Qu.: 5.188  
 Max.   :0.8710   Max.   :8.780   Max.   :100.00   Max.   :12.127  
      RAD              TAX 

La dataframe contient 506 lignes (chaque ligne représente une maison) et 13 variables. Pour
comprendre la signification de chaque variable, veuillez-vous reporter à la [page descriptive](https://raw.githubusercontent.com/JosueAfouda/Boston-Housing/master/housing.names)
de ces données. La variable ***MEDV*** est notre variable d’intérêt. Elle indique le prix
des maisons en millier de dollars.

Apparemment, il n’y a pas de valeurs manquantes dans la dataframe. 

# <font color=red> Données d'entraînement et d'évaluation (*Train/Test*)

In [5]:
# Mélange aléatoire des données

seed <- 123

set.seed(seed)

rows <- sample(nrow(boston))

boston_mel <- boston[rows, ]

head(boston_mel)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
415,45.74610,0,18.10,0,0.693,4.519,100.0,1.6582,24,666,20.2,36.98,7.0
463,6.65492,0,18.10,0,0.713,6.317,83.0,2.7344,24,666,20.2,13.99,19.5
179,0.06642,0,4.05,0,0.510,6.860,74.4,2.9153,5,296,16.6,6.92,29.9
14,0.62976,0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,8.26,20.4
195,0.01439,60,2.93,0,0.401,6.604,18.8,6.2196,1,265,15.6,4.38,29.1
426,15.86030,0,18.10,0,0.679,5.896,95.4,1.9096,24,666,20.2,24.39,8.3


In [6]:
# Division en données d'entraînement (80%) et Données d'évaluation (20%)

split <- round(nrow(boston_mel) * 0.80)

# Train data

train <- boston_mel[1:split, ]

# Test data

test <- boston_mel[(split + 1):nrow(boston_mel), ]

In [7]:
# Dimension du train data

dim(train)

[1] 405  13

In [8]:
# Dimension du test data

dim(test)

[1] 101  13

# <font color=red> Modèle et calcul de l'erreur quadratique moyenne

In [9]:
# Construction du modèle à partir des données d'entraînement

model <- lm(MEDV ~., data = train)

# Affichage du modèle

model


Call:
lm(formula = MEDV ~ ., data = train)

Coefficients:
(Intercept)         CRIM           ZN        INDUS         CHAS          NOX  
   43.92193     -0.11340      0.04921      0.01164      3.19107    -19.12204  
         RM          AGE          DIS          RAD          TAX      PTRATIO  
    3.32900      0.01363     -1.47708      0.30445     -0.01190     -0.96944  
      LSTAT  
   -0.61274  


Passons à l'évaluation de la performance du modèle. Il s'agit de vérifier la capacité du modèle sur des données qui n'ont pas servi à son entraînement c'est-à-dire sa capacité à se généraliser.

In [10]:
# Création d'une Fonction d'évaluation de modèle

model_evaluation <- function (Model) {
    
    # Prédiction sur le train data
    
    preds_train <- predict(Model, train)
    
    # Prédictions sur le test data
    
    preds_test <- predict(Model, test)
    
    # Erreur sur le train data
    
    error_train <- preds_train - train[, "MEDV"]
    
    # Erreur sur le test data
    
    error_test <- preds_test - test[, "MEDV"]
    
    # RMSE sur les train data
    
    rmse_train <- sqrt(mean(error_train ^ 2))
    
    # RMSE sur le test data
    
    rmse_test <- sqrt(mean(error_test ^ 2))
    
    print(paste("RMSE sur le train data :", rmse_train))
    
    print(paste('RMSE sur le test data :', rmse_test))
}

In [11]:
# Evaluation du modèle 'model'

model_evaluation(model)

[1] "RMSE sur le train data : 4.67603405287116"
[1] "RMSE sur le test data : 5.02971020858123"


# <font color=red> Validation croisée simple

En divisant nos données, nous avons pu évaluer la performance du modèle sur les données de test afin
de nous assurer que le modèle est capable de bien s’ajuster à de nouvelles données. Mais, il y a
un écueil dans ce processus. Les données de test peuvent avoir certaines particularités qui font
que l’erreur quadratique moyenne ait donné telle ou telle autre valeur. Par exemple, la présence ou l'absence d'une valeur aberrante peut modifier considérablement la valeur de l'erreur quadratique moyenne aussi bien au niveau de l'ensemble d'entraînement que de l'ensemble d'évaluation. 

Donc la métrique d'évaluation calculée suite à une simple division aléatoire arbitraire n’est pas forcément représentative de la capacité du modèle à généraliser sur de
nouvelles données. 

Une meilleure approche est l'utilisation de la technique de la **validation croisée** (***cross_validation***).

Pour comprendre la technique de cross-validation :

- Commençons par diviser la dataframe en 5 groupes ou plis ;

- Ensuite, le premier pli est pris comme ensemble de test et l’algorithme est entraîné avec les 4 plis restants. On fait les prédictions sur l’ensemble de test puis on calcule la métrique d’évaluation du modèle ;

- Le deuxième pli est maintenant utilisé comme ensemble de test et les 4 autres plis comme données d’entraînement ;

- On fait de même jusqu’à ce que chacun des 5 plis ait été utilisé comme ensemble de test.

Finalement, on se retrouve avec 5 modèles donc 5 métriques (*RMSE* par exemple) et on peut
calculer des statistiques comme la moyenne ou la médiane de ces métriques. Au lieu de 5 plis,
vous pouvez choisir *k* plis (***k-fold cross validation***) avec *k* un nombre entier naturel non null.

Mais attention au nombre de plis car plus vous avez de plis, plus cela est coûteux en termes de
calcul. La méthode de validation croisée nous permet d’éviter que la métrique d’évaluation
choisie ne dépende du fractionnement arbitraire de la fonction *sample()*.

Appliquons la technique de validation croisée avec 5 plis.

In [12]:
# Modèle de régression avec validation croisée

model_cv <- train(MEDV ~ ., 
                 data = train,
                 method = "lm",
                 trControl = trainControl(method = "cv",
                                          number = 5
                                         )
                 )

# Affichage du model

model_cv

Linear Regression 

405 samples
 12 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 324, 323, 324, 325, 324 
Resampling results:

  RMSE      Rsquared   MAE     
  4.876903  0.7166239  3.417444

Tuning parameter 'intercept' was held constant at a value of TRUE

In [13]:
# Evaluation du modèle 'model_cv'

model_evaluation(model_cv)

[1] "RMSE sur le train data : 4.67603405287116"
[1] "RMSE sur le test data : 5.02971020858123"


# <font color=red> Validation croisée répétée

Il est possible de faire plus qu'une seule itération de validation croisée. La validation croisée répétée donne une meilleure estimation de la performance du modèle.

In [14]:
# Modéle de régréssion avec répétition du processus de validation croisée

model_cv2 <- train(MEDV ~ ., 
                 data = train,
                 method = "lm",
                 trControl = trainControl(method = "repeatedcv",
                                          number = 5,
                                          repeats = 5
                                         )
                 )

# Affichage du modèle

model_cv2

Linear Regression 

405 samples
 12 predictor

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 5 times) 
Summary of sample sizes: 325, 325, 324, 323, 323, 323, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  4.867522  0.7276673  3.416793

Tuning parameter 'intercept' was held constant at a value of TRUE

In [15]:
# Evaluation du modèle 'model_repeatcv'

model_evaluation(model_cv2)

[1] "RMSE sur le train data : 4.67603405287116"
[1] "RMSE sur le test data : 5.02971020858123"


# <font color=red> Modèle sur des données prétraitées

Bien que les données ne contiennent pas de valeurs manquantes, elles nécessites quand même un petit prétraitement avat modélisation. En effet, les données ne sont pas à la même échelle et cela pourrait causer une mauvaise performance du modèle. 

L'argument ***preProcess*** de la fonction [train](https://www.rdocumentation.org/packages/caret/versions/4.47/topics/train) permet d'opérer très facilement plusieurs étapes de prétraitement des données avent modélisation.

In [16]:
# Standardisation des données avant modélisation


model_stand <- train(MEDV ~ ., 
                 data = train,
                 method = "lm",
                 PreProcess = c("center", "scaler"),
                 trControl = trainControl(method = "cv",
                                          number = 5
                                         )
                 )

# Affichage du modèle

model_stand

Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"

Linear Regression 

405 samples
 12 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 324, 324, 324, 324, 324 
Resampling results:

  RMSE      Rsquared   MAE     
  4.841974  0.7341165  3.421173

Tuning parameter 'intercept' was held constant at a value of TRUE

In [17]:
# Evaluation du modèle 'model_stand'

model_evaluation(model_stand)

[1] "RMSE sur le train data : 4.67603405287116"
[1] "RMSE sur le test data : 5.02971020858123"


In [18]:
# Standardisation et ACP des données avant modélisation

model_pca <- train(MEDV ~ ., 
                 data = train,
                 method = "lm",
                 PreProcess = c("center", "scaler", "pca"),
                 trControl = trainControl(method = "cv",
                                          number = 5
                                         )
                 )

# Affichage du modèle

model_pca

Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"Warning message:
"In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument 'PreProcess' will be disregarded"

Linear Regression 

405 samples
 12 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 324, 324, 325, 324, 323 
Resampling results:

  RMSE      Rsquared   MAE     
  4.909677  0.7154149  3.465592

Tuning parameter 'intercept' was held constant at a value of TRUE

In [19]:
# Evaluation du modèle 'model_pca'

model_evaluation(model_pca)

[1] "RMSE sur le train data : 4.67603405287116"
[1] "RMSE sur le test data : 5.02971020858123"
